## Function Calling

Read:

https://platform.openai.com/docs/guides/function-calling

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [ ]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_: bool = load_dotenv(find_dotenv())      # read local .env file

client: OpenAI = OpenAI()

## The basic sequence of steps for function calling is as follows:

1. Call the model with the user query and a set of functions defined in the functions parameter.

2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).

3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.

4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

<img src="../../assets/first.png" alt="process describe of function calling in genai"/>

https://www.linkedin.com/pulse/azure-openai-function-calling-tarun-sharma/

In [ ]:
# example dummy function hard coded to return the same weather

# in production this could be your backend api or an external api

def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({ "location": "Tokyo", "temperature": "10", "unit": "celsius" })
    elif "san francisco" in location.lower():
        return json.dumps({ "location": "san francisco", "temperature": "72", "unit": "fahrenheit" })
    elif "paris" in location.lower():
        return json.dumps({ "location": "paris", "temperature": "22", "unit": "celsius" })
    else:
        return json.dumps({ "location": location, "temperature": "unknown" }) 
    
    

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Types

https://github.com/openai/openai-python/tree/main/src/openai/types/chat

In [1]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion

def run_conversation(main_resquest: str) -> str:
    
    # step : 01 send the conversation and available functions to the model
    
    messages: [ { "role": "user", "content": main_resquest } ]
    tools: [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                                "type": "string",
                                "description": "The city and state eg. San Francisco, CA"
                            },
                        "unit": {
                            "type": "string",
                            "enum": ["celsius", "fahranheit"]
                        }
                    },
                },
                "required": ["location"]
            }
        }
    ]
    
    # first request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto"     # auto is default, but we'll be explicit
    )
    
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* first response, ", dict[response_message])
    
    tool_calls = response_message.tool_calls
    display("* first response of tool calls, ", list[tool_calls])
    
    
    # step : 02 if the model wanted to call a function
    if tool_calls:
        
        # step : 03 call the function
        
        # note : the json response may not always be valid; be sure to handle errors
        
        available_functions = {
            "get_current_weather": get_current_weather
            
            # only one function in this example, but you have multiple
        }
        
        messages.append(response_message)     # extends conversation with assistent's reply
        
        
        # step : 04 send the info for each function call and function response to the model
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name,
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get('location'),
                unit=function_args.get("unit")
            )
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                }
            )          # extends conversation with function response
            
            second_response: ChatCompletion = client.chat.completions.create(
                messages=messages,
                model="gpt-3.5-turbo-1106",
            )     # get a new response from the model where it can see the function response
            
            print("* second response, ", dict[second_response])
            
            return second_response.choices[0].message.content

In [2]:
run_conversation("What's the weather like in San Francisco, Tokyo and Paris")

NameError: name 'client' is not defined